# IMPORT

In [ ]:
import shutil
shutil.rmtree('cultural_classification_nlp', ignore_errors=True)

!git clone "https://github.com/cybernetic-m/cultural_classification_nlp.git"
!pip install --quiet wikipedia wikidata datasets 2> /dev/null    # the 2> /dev/null redirects standard error to the null avoiding the print of dependencies conflicts

# Import from our GitHub Repository of needed functions and classes for the LM-based evaluation
from cultural_classification_nlp.models.LMClassifier import LMClassifier
from cultural_classification_nlp.testing.test import eval_lm
from cultural_classification_nlp.graph.src.test import test, eval_non_lm

# NN import
import torch
import pandas as pd
from huggingface_hub import hf_hub_download
from transformers import (
    AutoModel,
    AutoTokenizer,
    set_seed,
)

set_seed(10) # Set the seed for reproducibility

device ='cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Cloning into 'cultural_classification_nlp'...
remote: Enumerating objects: 627, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 627 (delta 77), reused 107 (delta 39), pack-reused 468 (from 1)
Receiving objects: 100% (627/627), 10.45 MiB | 3.19 MiB/s, done.
Resolving deltas: 100% (310/310), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.2 MB/s eta 0:00:00


# Evaluation non LM-based model

**Load of the dataset**

**⚠️TAs Instructions⚠️**:

1. Upload the test dataset as a csv file into the "*Files*" space of Colab

2. Write the name of the dataset into the variable *dataset_name*

In [ ]:
dataset_name = 'test_unlabeled.csv'
dataset_path = './' + dataset_name

The eval_non_lm function returns the final annotated dataframe with columns qid and label (predicted). This function will receive the the test dataframe that will be processed in order to extract the properties for each item and the visualization of the wikipedia pages associated for 100 languages. It also performs the encoding of the category and subcategory.
After the pre processing the function adds the informations to a graph that is pre-loaded with the training data. The test data will be added to the graph with label -1 (to be predicted), then the label propagation algorithm will be applied on the graph with knn and 10 neighbors.
The function will also save the dataframe returned as a csv file in the notebook files.

In [ ]:
 # this does everything, is sufficient to run the test set and get the final df
 # the print_statistics will print the confusion matrix and classification report if you submit a df with labels
predictions, df_ex = eval_non_lm(dataset_path, path = './cultural_classification_nlp/graph/models')

# Evaluation LM-based model

**Load of the dataset**

Instructions:
1. Upload the test dataset as a csv file into the "*Files*" space of Colab

2. Write the name of the dataset into the variable *dataset_name*



In [ ]:
dataset_name = 'test_unlabeled.csv'
batch_size = 128
max_length = 64
dataset_path = './' + dataset_name

**Definition of the model**


In [ ]:
# Definition of the encoder
encoder = AutoModel.from_pretrained('microsoft/deberta-v3-base')
tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-base')

# Definition of the classifier
model = LMClassifier(
    encoder=encoder,
    num_labels=3,
    mlp_list=[128],
    classifier_type='mlp',
    pooling= 'attention'
).to(device)

model_path = hf_hub_download(repo_id = 'cybernetic-m/deBERTa_cultural_classifier', filename = 'model.pt')

Encoder type:

 DebertaV2Model(
  (embeddings): DebertaV2Embeddings(
    (word_embeddings): Embedding(128100, 768, padding_idx=0)
    (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): DebertaV2Encoder(
    (layer): ModuleList(
      (0-11): 12 x DebertaV2Layer(
        (attention): DebertaV2Attention(
          (self): DisentangledSelfAttention(
            (query_proj): Linear(in_features=768, out_features=768, bias=True)
            (key_proj): Linear(in_features=768, out_features=768, bias=True)
            (value_proj): Linear(in_features=768, out_features=768, bias=True)
            (pos_dropout): Dropout(p=0.1, inplace=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): DebertaV2SelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-07, elementwise_affine=True)
            (dropout):

In [ ]:
test_metrics_dict = {}
loss_fn = torch.nn.CrossEntropyLoss()

eval_lm (
    model=model,
    model_path=model_path,
    dataset_csv = dataset_path,
    tokenizer=tokenizer,
    batch_size=batch_size,
    max_length=max_length,
    test_metrics_dict=test_metrics_dict,
    loss_fn=loss_fn,
    device=device
)

Adding Wikipedia data...:   0%|          | 0/300 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



DisambiguationError for 'imam': options are ['Mother', 'Mama-san', 'uncle']...

No page found for 'empiricism'

DisambiguationError for 'Alnus': options are ['Alto', 'Altus (voice type)', 'Altus, Arkansas']...

No page found for 'David Ginola'

No page found for 'beIN SPORTS'

DisambiguationError for 'toast': options are ['Töss', 'Winterthur Töss railway station', 'Töss Monastery']...

DisambiguationError for 'escalator': options are ['Cost escalation', 'Conflict escalation', 'Escalation hypothesis']...

No page found for 'bitter lemon'

No page found for 'singer-songwriter'

DisambiguationError for 'Open Season 2': options are ['Open season (hunting)', 'Annual enrollment', 'Open Season (1974 film)']...

No page found for '740 Park Avenue'

DisambiguationError for 'nori': options are ['Noricum', 'Noric Alps', 'Noric language']...

No page found for 'diplomatician'

No page found for 'singer'

No page found for 'Studio Ghibli'

DisambiguationError for 'Crackle': options are ['Cradle (b